In [1]:
from bs4 import BeautifulSoup
import requests
import webbrowser
import mechanize
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import numpy as np
import csv
from openpyxl import Workbook
from openpyxl import load_workbook

In [2]:
page_link="https://atlantatechvillage.com/community/current-villagers/"
hdr = {'User-Agent': 'Mozilla/5.0'}
page_resp=requests.get(page_link, headers=hdr)
page_content=BeautifulSoup(page_resp.content, "html.parser")

In [4]:
def filter_func(s, bw):
    return "http" in s and not any(x in s for x in bw) and not None

def remove_bad_links(links):
    bad_words=["atlantatechvillage", "instagram", "spotify", "youtube", "twitter", "facebook"]
    good_links=[]
    good_index=[]
    bad_index=[]
    for i, link in enumerate(links):
        try:
            if filter_func(link, bad_words):
                print("good link: ", link)
                good_links.append(link)
                good_index.append(i)
            else:
                print("didnt work booo: ", link)
                continue
        except:
            print("Maybe Nonetype?: ", link, i)
            bad_index.append(i)
            continue
    
    return good_links, good_index, bad_index


def bad_p(l, string):
    bad_index=l.index(string)
    del l[bad_index]
    return l
    

In [5]:
links=page_content.find_all("a")
descriptions=page_content.find_all("p")

link_df=pd.DataFrame()
comp_links=[]
good_links, good_index, bad_index=remove_bad_links([link.get("href") for link in links])

fake_desc=[desc.get_text() for desc in descriptions]


good_desc2=bad_p(fake_desc, "We Make Marketing Simple")

link_desc={}

didnt work booo:  /members/signin/
didnt work booo:  https://atlantatechvillage.com/
didnt work booo:  https://atlantatechvillage.com/
didnt work booo:  https://atlantatechvillage.com/community/
didnt work booo:  https://atlantatechvillage.com/about/
didnt work booo:  https://atlantatechvillage.com/events/
didnt work booo:  https://atlantatechvillage.com/programs/
didnt work booo:  https://atlantatechvillage.com/buzz/
didnt work booo:  https://atlantatechvillage.com/join/
didnt work booo:  https://atlantatechvillage.com
didnt work booo:  https://atlantatechvillage.com/community/
didnt work booo:  https://atlantatechvillage.com/community/current-villagers/
didnt work booo:  https://atlantatechvillage.com/community/alumni/
didnt work booo:  https://atlantatechvillage.com/community/mentors/
didnt work booo:  https://atlantatechvillage.com/join/sponsors/
didnt work booo:  https://atlantatechvillage.com/community/about-us/job-board/
good link:  http://www.acuitycomplete.com/
good link:  htt

In [6]:
del good_desc2[28]
del good_desc2[61]
del good_desc2[127:]


In [7]:
if len(good_links)!=len(good_desc2):
    print("Bad length matching")

i=0
for link, desc in zip(good_links, good_desc2[:127]):
    link_df.loc[i, "link"]=link
    link_df.loc[i, "Description"]=desc
    i+=1

127 127


In [10]:
pd.options.display.max_rows=200

link_df["City"]="Atlanta"

link_df.columns=["Link", "Company Info", "City"]

link_df["Incubator/VC"]="Incubator"
link_df["Company"]=np.nan

full_df=link_df[["City", "Incubator/VC", "Company", "Company Info", "Link"]]

full_df.head()

,City,Incubator/VC,Company,Company Info,Link
0,Atlanta,Incubator,NaN,AcuityComplete is remote bookkeeping for entre...,http://www.acuitycomplete.com/
1,Atlanta,Incubator,NaN,The cornerstone of the approach is our DNA and...,https://apogeelegal.com
2,Atlanta,Incubator,NaN,AppZoro is custom Mobile Apps Development stud...,https://www.appzoro.com/
3,Atlanta,Incubator,NaN,Aprio is a premier CPA-led business advisory f...,http://www.aprio.com/
4,Atlanta,Incubator,NaN,Business Appraisers and Strategic Advisors – O...,http://www.arpeggioadvisors.com/


In [11]:
full_df.to_csv(r"C:\Users\andyk\OneDrive\Documents\Untitled Folder\Atl_cos.csv", index=False, header=False)